In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pickle

Mounted at /content/drive


In [2]:
file = open('/content/drive/MyDrive/kaggle/covid_dataset.pkl', 'rb')
checkpoint = pickle.load(file)
file.close()
X_train, y_train, X_val, y_val, X_test = checkpoint["X_train"], checkpoint["y_train_log_pos_cases"], checkpoint["X_val"], checkpoint["y_val_log_pos_cases"], checkpoint["X_test"]
import pandas as pd 
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler
# save the prediction array test_pred into a valid submission file

In [3]:
def fill_na_and_normalize(sparse):
  sparse = pd.DataFrame(sparse)
  sparse = sparse.fillna(value = np.nan)

  imputer = IterativeImputer(random_state = 0, max_iter = 1000)

  sparse = imputer.fit_transform(sparse)

  sparse = pd.DataFrame(sparse)

  scaler = MinMaxScaler(feature_range = (1,10))

  scaler.fit(sparse)

  sparse = scaler.transform(sparse)

  return sparse

In [4]:
X_train_f = fill_na_and_normalize(X_train)
X_test_f = fill_na_and_normalize(X_test)
X_val_f = fill_na_and_normalize(X_val)

In [5]:
def model_error(model):
  print(cross_val_score(model, X_val_f, y_val, cv=5, scoring = 'neg_mean_squared_error').mean())
  print(np.sum((model.predict(X_val_f) - y_val) ** 2))

In [6]:
krr_GBR = GradientBoostingRegressor(random_state = 0, n_estimators = 1000, learning_rate = .1, min_samples_leaf=4)
krr_GBR.fit(X_train_f, y_train)

model_error(krr_GBR)

-3.873500780938869
95.51937811636563


ready!

In [7]:
test_pred = krr_GBR.predict(X_test_f)

data = pd.DataFrame(test_pred).to_csv("predictions.csv", header=["cases"], index_label="id")